In [72]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.decomposition import SparsePCA
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB,GaussianNB,MultinomialNB
from sklearn.pipeline import Pipeline, FeatureUnion
import matplotlib.pyplot as plt
import itertools

In [2]:
text_data=[]
languages=[]

with open("data/train.txt", mode='r',encoding="utf-8") as f:
    line = f.readline()
    while line:
        sentence,language = line.split("\t")
        text_data.append(sentence)
        languages.append(language[:-1])
        line = f.readline()


In [54]:
df = pd.DataFrame()
df["text"]=text_data
df["lang"]=languages
cv = CountVectorizer()
X = cv.fit_transform(df["text"])
y = df["lang"]




In [4]:
lda = TruncatedSVD(n_components=25)
kbest = SelectKBest(chi2,k=25)
combined_features = FeatureUnion([("svd",lda),("chi2",kbest)])
X_features = combined_features.fit_transform(X,y)


In [13]:
# 100 features were taken as max , since with 200 alot of ram and resources were being consumed
lda10 = TruncatedSVD(n_components=10)
lda50 = TruncatedSVD(n_components=50)
lda100 = TruncatedSVD(n_components=100)


In [14]:
x_10 = lda10.fit_transform(X,y)
x_50 = lda50.fit_transform(X,y)
x_100 = lda100.fit_transform(X,y)
x_200 = lda200.fit_transform(X,y)

10

In [ ]:
import seaborn as sns; sns.set(style="ticks", color_codes=True)
x=lda10.fit_transform(X,y)
df = pd.DataFrame(x)
df["target"]=y
df_shuffle = df.sample(frac=1)
df1 = df_shuffle.iloc[:100,:-1]
# target = df_shuffle["target"]
g = sns.pairplot(df1, palette="husl",kind="reg")
plt.savefig("LDA_pairwise_visualization.png")
plt.show()


In [67]:
x=SelectKBest(chi2,k=10).fit_transform(X,y)
# df = pd.DataFrame(x)
# df["target"]=y
# df_shuffle = df.sample(frac=1)
# df1 = df_shuffle.iloc[:100,:-1]
# # target = df_shuffle["target"]
# g = sns.pairplot(df1, palette="husl",kind="reg")
# plt.savefig("Kbest_pairwise_visualization.png")
# plt.show()

ValueError: DataFrame constructor not properly called!

['bg', 'bg']


array(['bg', 'bs', 'cz', 'es-AR', 'es-ES', 'hr', 'id', 'mk', 'my',
       'pt-BR', 'pt-PT', 'sk', 'sr', 'xx'], dtype='<U5')

,text,lang
count,252000,252000
unique,252000,14
top,David Schwimmer se svou o dvacet let mladší př...,sr
freq,1,18000


In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    plt.figure(figsize=(15,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')



In [ ]:
# SVC is not good for more than 10k rows
# Only the number of features taken 100
x_svc = df_shuffle.iloc[:100,:-1]
y_svc = target[:100]
svc = SVC()
svc.fit(x_svc,y_svc)
accuracy_score(y_svc,svc.predict(x_svc))
conf_matrx = confusion_matrix(y_svc,svc.predict(x_svc))
plot_confusion_matrix(conf_matrx,classes=np.unique(y),normalize=True,title = "SVC Regression Conf_Matrix")
plt.savefig("SVC_confustion_matrix.png")
plt.show()


ValueError: X has 10 features per sample; expecting 100

In [22]:
# select the data to train on : x_10, x_50, x_100 , X_features
new_X = x_10


In [21]:
lr = LogisticRegression()
lr.fit(new_X,y)
print(accuracy_score(y,lr.predict(new_X)))
conf_matrx = confusion_matrix(y,lr.predict(new_X))
plot_confusion_matrix(conf_matrx,classes=np.unique(y),normalize=True,title = "Logistic Regression Conf_Matrix")
plt.savefig("LR_confustion_matrix.png")
plt.show()


0.7769246031746032


TypeError: 'conf_matrx' is an invalid keyword argument for this function

In [ ]:

rf = RandomForestClassifier()
rf.fit(new_X,y)
print(accuracy_score(y,rf.predict(new_X)))
conf_matrx = confusion_matrix(y,rf.predict(new_X))
plot_confusion_matrix(conf_matrx,classes=np.unique(y),normalize=True,title = "Random Forest Conf_Matrix")
plt.savefig("RF_confustion_matrix.png")
plt.show()


0.9999960317460318



DT SCORE :
wiht combined_Features 50 : 98%
with 10 lda features : 99%
with lda 50 lda features : 99%
with lda 100 lda features : too much time taken 


In [75]:

dt = DecisionTreeClassifier()
dt.fit(new_X,y)
print(accuracy_score(y,dt.predict(new_X)))
conf_matrx = confusion_matrix(y,rf.predict(new_X))
plot_confusion_matrix(conf_matrx,classes=np.unique(y),normalize=True,title = "DecissionTree Conf_Matrix")
plt.savefig("DT_confustion_matrix.png")
plt.show()



0.9999960317460318


In [27]:

bernouli = BernoulliNB(alpha=1.0, binarize=0.0,class_prior=None, fit_prior=True)
bernouli.fit(new_X,y)
print(accuracy_score(y,bernouli.predict(new_X)))
# print(confusion_matrix(y,bernouli.predict(new_X)))
conf_matrx = confusion_matrix(y,bernouli.predict(X_features))
plot_confusion_matrix(conf_matrx,classes=np.unique(y),normalize=True,title = "Bernoulli Conf_Matrix")
plt.savefig("Bernoulli_confustion_matrix.png")
plt.show()

0.617563492063492


In [ ]:
# change k for varying results : k = 10,50,100
k_features = SelectKBest(chi2,k=100)
kbest_mn = k_features.fit_transform(X,y)
new_X = kbest_mn
multinomail = MultinomialNB()
multinomail.fit(new_X,y)
accuracy_score(y,multinomail.predict(new_X))
conf_matrx = confusion_matrix(y,multinomail.predict(new_X))
plot_confusion_matrix(conf_matrx,classes=np.unique(y),normalize=True,title = "Multinomial Conf_Matrix")
plt.savefig("MulinomialNB _confustion_matrix.png")
plt.show()


In [44]:
X = df["text"]
y = df["lang"]
pipe_bernoulli = Pipeline([('count_vec',cv),
                 ('combined_features',temp_X),('gaussian',bernouli)])
pipe_bernoulli.fit(X,y)
y_pred = pipe_bernoulli.predict(X)
conf_matrx = confusion_matrix(y,y_pred)
plot_confusion_matrix(conf_matrx,classes=np.unique(y),normalize=True,title = "BernouliNB with pipe Conf_Matrix")
plt.savefig("Bernoulli with pipe confusion matrix.png")
plt.show()

Normalized confusion matrix


In [50]:
X = df["text"]
y = df["lang"]
pipe_multinomial = Pipeline([('count_vec',cv),
                 ('kbest',k_features),('multinomial',multinomail)])
pipe_multinomial.fit(X,y)
y_pred = pipe_multinomial.predict(X)
conf_matrx = confusion_matrix(y,y_pred)
plot_confusion_matrix(conf_matrx,classes=np.unique(y),normalize=True,title = "MultinomialNB with pipe Conf_Matrix")
plt.savefig("Multinomial with pipe confusion matrix.png")
plt.show()

Normalized confusion matrix


In [ ]:
X = df["text"]
y = df["lang"]
pipe_rf= Pipeline([('count_vec',cv),
                 ('combined_features',combined_features),('RF',rf)])
pipe_rf.fit(X,y)
y_pred = pipe_rf.predict(X)
conf_matrx = confusion_matrix(y,y_pred)
plot_confusion_matrix(conf_matrx,classes=np.unique(y),normalize=True,title = "RF with pipe Conf_Matrix")
plt.savefig("RF with pipe confusion matrix.png")
plt.show()


In [ ]:
X = df["text"]
y = df["lang"]
pipe_dt= Pipeline([('count_vec',cv),
                 ('combined_features',combined_features),('DT',dt)])
pipe_dt.fit(X,y)
y_pred = pipe_dt.predict(X)
conf_matrx = confusion_matrix(y,y_pred)
plot_confusion_matrix(conf_matrx,classes=np.unique(y),normalize=True,title = "DT with pipe Conf_Matrix")
plt.savefig("DT with pipe confusion matrix.png")
plt.show()




In [17]:
X = df["text"]
y = df["lang"]
pipe_lr= Pipeline([('count_vec',cv),
                 ('combined_features',combined_features),('LR',lr)])
pipe_lr.fit(X,y)
y_pred = pipe_lr.predict(X)
conf_matrx = confusion_matrix(y,y_pred)
plot_confusion_matrix(conf_matrx,classes=np.unique(y),normalize=True,title = "LR with pipe Conf_Matrix")
plt.savefig("LR with pipe confusion matrix.png")
plt.show()

In [79]:
classifiers = ("LR","RF","DT","Bernouli NB","Multinomial NB")
y_pos = np.arange(len(classifiers))
performance = [71,99,99,61,63]
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Accuracy')
plt.title('Classifiers')
plt.savefig("Classifier Accuracy Comparison.png")
plt.show()


In [73]:
pipe.named_steps

{'combined_features': FeatureUnion(n_jobs=1,
        transformer_list=[('svd', TruncatedSVD(algorithm='randomized', n_components=25, n_iter=5,
        random_state=None, tol=0.0)), ('chi2', SelectKBest(k=25, score_func=<function chi2 at 0x0000022B8F6262F0>))],
        transformer_weights=None),
 'count_vec': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), preprocessor=None, stop_words=None,
         strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
         tokenizer=None, vocabulary=None),
 'gaussian': BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True),
 'tfidf': TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)}

In [81]:
pipe.named_steps.combined_features__transformer_list=[('svd', TruncatedSVD(algorithm='randomized', n_components=25, n_iter=5,
        random_state=None, tol=0.0),('chi2', SelectKBest(chi2 ,k=25))),('svd', TruncatedSVD(algorithm='randomized',
                                                                                            n_components=50, n_iter=5,
        random_state=None, tol=0.0),('chi2', SelectKBest(chi2 ,k=500)))]


In [83]:
pipe.named_steps.gaussian__alpha = [1.0,0.8,0.5]
pipe.named_steps.tfidf__sublinear=[True,False]

In [94]:
param_grid=dict(combined_features__transformer_list=[[('svd', TruncatedSVD(algorithm='randomized', n_components=25, n_iter=5,
        random_state=None, tol=0.0)),('chi2', SelectKBest(chi2 ,k=25))],[('svd', TruncatedSVD(algorithm='randomized',
                                                                                            n_components=50, n_iter=5,
        random_state=None, tol=0.0)),('chi2', SelectKBest(chi2 ,k=100))]],
                gaussian__alpha = [1.0,0.8,0.5],
                tfidf__sublinear_tf=[True,False]
)

In [95]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(pipe, param_grid=param_grid)


In [96]:
grid_search.fit(X,y)